# Implementing RRT Connect (6 pts)

In [1]:
from Practical04_Support.Obstacle import *
from Practical04_Support.path_animation import *
import numpy as np
import random
import os
import types

# Import dependencies and set random seed
seed_value = 5
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

Taking the code of the standard RRT algorithm as a starting point, you are asked to implement a bi-directionnal tree roadmap planning algorithm (reference pseudocode shown below)

![title](https://i.postimg.cc/HsR6btP8/rrt-pseudo.png)

The following ``RRTC`` class encapsulates the algorithm.

Your task is to complete the class methods marked with TODO:
- ``grow_tree`` (2 points)
- ``check_trees_distance`` (1 point)
- ``planning`` (3 points)

1. Sample and add a node ``new_node`` to the start tree (``self.start_node_list``)
2. Check if there is a node in the end tree (``self.end_node_list``) that is at a distance lesser than ``expand_dis`` from ``new_node``. You can use the existing function ``get_nearest_node_index(.)`` to do so.
3. If that is the case, the trees can be connected and a path has been found! To generate the path, add ``new_node`` to the end tree (after checking that there are no collisions) and return the path generated by the ``self.generate_final_course(len(self.start_node_list) - 1, len(self.end_node_list) - 1))``. The path is ready to be displayed.
4. If there is no node in the end tree close to ``new_node``, sample and add a node to the end tree. 
5. Swap start and end trees.

You can access the other ``RRTC`` class methods to complete your task.


In [2]:
class RRTC:
    """
    Class for RRT planning
    """
    class Node:
        """
        RRT Node
        """
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.path_x = []
            self.path_y = []
            self.parent = None

        def __eq__(self, other):
            bool_list = []
            bool_list.append(self.x == other.x)
            bool_list.append(self.y == other.y)
            bool_list.append(np.all(np.isclose(self.path_x, other.path_x)))
            bool_list.append(np.all(np.isclose(self.path_y, other.path_y)))
            bool_list.append(self.parent == other.parent)
            return np.all(bool_list)

    def __init__(self, start=np.zeros(2),
                 goal=np.array([120,90]),
                 obstacle_list=None,
                 width = 160,
                 height=100,
                 expand_dis=3.0, 
                 path_resolution=0.5, 
                 max_points=200):
        """
        Setting Parameter
        start:Start Position [x,y]
        goal:Goal Position [x,y]
        obstacle_list: list of obstacle objects
        width, height: search area
        expand_dis: min distance between random node and closest node in rrt to it
        path_resolution: step size to considered when looking for node to expand
        """
        self.start = self.Node(start[0], start[1])
        self.end = self.Node(goal[0], goal[1])
        self.width = width
        self.height = height
        self.expand_dis = expand_dis
        self.path_resolution = path_resolution
        self.max_nodes = max_points
        self.obstacle_list = obstacle_list
        self.start_node_list = [] # Tree from start
        self.end_node_list = [] # Tree from end

    def grow_tree(self, tree, node):
        # Extend a tree towards a specified node, starting from the closest node in the tree,
        # and return a Bolean specifying whether we should add the specified node or not
        # `added_new_node` is the Boolean.
        # If you plan on appending a new element to the tree, you should do that inside this function
        
        # Extend the tree
        nearest_index = self.get_nearest_node_index(tree,node)
        nearest_node = tree[nearest_index]
        new_node = self.steer(nearest_node, node, self.expand_dis)

        # Check if we should add this node or not, and add it to the tree
        added_new_node = False
        new_index = False

        if self.is_collision_free(new_node):
            tree.append(new_node)
            added_new_node = True
            new_index = len(tree) - 1 
        
        return added_new_node, new_index

    def check_trees_distance(self):
        # Find the distance between the trees, return if the trees distance is smaller than self.expand_dis
        # In other word, we are checking if we can connect the 2 trees.
        
        can_be_connected = False
        for start_node in self.start_node_list:
            for end_node in self.end_node_list:
                distance, _ = self.calc_distance_and_angle(start_node, end_node)
                if distance <= self.expand_dis:
                    can_be_connected = True
        
        return can_be_connected

    def planning(self):
        """
        rrt path planning
        """
        
        self.start_node_list = [self.start]
        self.end_node_list = [self.end]
        while len(self.start_node_list) + len(self.end_node_list) <= self.max_nodes:
            
            # 1. Sample and add a node in the start tree
            rnd_node = self.get_random_node()
            added, start_new_idx = self.grow_tree(self.start_node_list,rnd_node)

            if added:
                new_node = self.start_node_list[start_new_idx]
            
                # 2. Check whether trees can be connected
                end_near_idx = self.get_nearest_node_index(self.end_node_list, new_node)
                end_near_node = self.end_node_list[end_near_idx]
                dist,_ = self.calc_distance_and_angle(end_near_node,new_node)

            if dist<= self.expand_dis:
                connector = self.steer(end_near_node,new_node,dist)
                # 3. Add the node that connects the trees and generate the path
                if self.is_collision_free(connector):
                    self.end_node_list.append(connector)
                    return self.generate_final_course(len(self.start_node_list) - 1, len(self.end_node_list) - 1)
            # 4. Sample and add a node in the end tree
            rnd_node = self.get_random_node()
            self.grow_tree(self.end_node_list, rnd_node)
            
            # 5. Swap start and end trees
            self.start_node_list, self.end_node_list = self.end_node_list, self.start_node_list
            
        return None  # cannot find path
    
    # ------------------------------DO NOT change helper methods below ----------------------------
    def steer(self, from_node, to_node, extend_length=float("inf")):
        """
        Given two nodes from_node, to_node, this method returns a node new_node such that new_node 
        is “closer” to to_node than from_node is.
        """
        
        new_node = self.Node(from_node.x, from_node.y)
        d, theta = self.calc_distance_and_angle(new_node, to_node)
        cos_theta, sin_theta = np.cos(theta), np.sin(theta)

        new_node.path_x = [new_node.x]
        new_node.path_y = [new_node.y]

        if extend_length > d:
            extend_length = d

        # How many intermediate positions are considered between from_node and to_node
        n_expand = math.floor(extend_length / self.path_resolution)

        # Compute all intermediate positions
        for _ in range(n_expand):
            new_node.x += self.path_resolution * cos_theta
            new_node.y += self.path_resolution * sin_theta
            new_node.path_x.append(new_node.x)
            new_node.path_y.append(new_node.y)

        d, _ = self.calc_distance_and_angle(new_node, to_node)
        if d <= self.path_resolution:
            new_node.path_x.append(to_node.x)
            new_node.path_y.append(to_node.y)

        new_node.parent = from_node

        return new_node

    def is_collision_free(self, new_node):
        """
        Determine if nearby_node (new_node) is in the collision-free space.
        """
        if new_node is None:
            return True
        
        points = np.vstack((new_node.path_x, new_node.path_y)).T
        for obs in self.obstacle_list:
            in_collision = obs.is_in_collision_with_points(points)
            if in_collision:
                return False
        
        return True  # safe
    
    def generate_final_course(self, start_mid_point, end_mid_point):
        """
        Reconstruct path from start to end node
        """
        # First half
        node = self.start_node_list[start_mid_point]
        path = []
        while node.parent is not None:
            path.append([node.x, node.y])
            node = node.parent
        path.append([node.x, node.y])
        
        # Other half
        node = self.end_node_list[end_mid_point]
        path = path[::-1]
        while node.parent is not None:
            path.append([node.x, node.y])
            node = node.parent
        path.append([node.x, node.y])

        return path

    def calc_dist_to_goal(self, x, y):
        dx = x - self.end.x
        dy = y - self.end.y
        return math.hypot(dx, dy)

    def get_random_node(self):
        x = self.width * np.random.random_sample()
        y = self.height * np.random.random_sample()
        rnd = self.Node(x, y)
        return rnd

    @staticmethod
    def get_nearest_node_index(node_list, rnd_node):        
        # Compute Euclidean disteance between rnd_node and all nodes in tree
        # Return index of closest element
        dlist = [(node.x - rnd_node.x) ** 2 + (node.y - rnd_node.y)
                 ** 2 for node in node_list]
        minind = dlist.index(min(dlist))
        return minind

    @staticmethod
    def calc_distance_and_angle(from_node, to_node):
        dx = to_node.x - from_node.x
        dy = to_node.y - from_node.y
        d = math.hypot(dx, dy)
        theta = math.atan2(dy, dx)
        return d, theta

## Test your Implementation

Use the code provided below to visually test your implementation.

**For grading** please make sure that you only work within the *TODO* block.

If your solution is correct you should obtain a path similar to this the first time you run ``animate_path_rrtc``.

<img src="https://i.postimg.cc/dty2cc4D/jlNe6Oq.gif" height="400" width="600" align="center">

In [6]:
import numpy as np
import plotly.graph_objects as go

# Set parameters
goal = np.array([14.0, 1.0])
start = np.array([1.0, 1.0])

all_obstacles = [Circle(11.5, 5, 2), Circle(4.5, 2.5, 2),
                 Circle(4.8, 8, 2.5)]

# start = np.array([1.0, 10.0])
# goal = np.array([14.0, 1.0])
# # expand_dis = 2.0
# all_obstacles = [Circle(11.5, 5, 3), Circle(6.5, 2.5, 2.5), Circle(4.8, 8, 3.0)]
# seed_value = 42
# os.environ['PYTHONHASHSEED'] = str(seed_value)
# random.seed(seed_value)
# np.random.seed(seed_value)

rrtc = RRTC(start=start, goal=goal, width=16, height=10, obstacle_list=all_obstacles,
            expand_dis=2.0, path_resolution=1)

animate_path_rrtc(rrtc)

# Grading

The test case will evaluate 3 functions this week: `RRTC.planning(), RRTC.check_trees_distance()` ,and `RRTC.grow_tree()`
This is just dummy test case, we will use a different combination of `start`, `goal`, `expand_dis`, and `all_obstacles` in the true grading (with specific seeding). Note that this exercise is also very prone to Infinite Loop so if you encounter one infinite loop among the 3 cases, you will get zero for the whole exercise.

In [4]:
import otter
from ece4078.Utility import pretty_print_otter
grader = otter.Notebook(tests_dir = "Practical04_Support/hidden_tests")

grader.check_all()
# If the test cases gives you too much messages and it's hard to keep track of, uncomment the following line
# pretty_print_otter(grader.check_all())

grow_tree results: All test cases passed!
grow_tree - 1 message: Correct implementation for adding node
grow_tree - 2 message: Correctly checking for collision

check_trees_distance results: All test cases passed!
check_trees_distance - 1 message: Correct implementation for distance checking

rrtc results:
    rrtc - 1 message: Failed Hidden Scenario 1

    rrtc - 1 result:
        Trying:
            import sys, os, random
        Expecting nothing
        ok
        Trying:
            import numpy as np
        Expecting nothing
        ok
        Trying:
            start = np.array([1.0, 10.0])
        Expecting nothing
        ok
        Trying:
            goal = np.array([14.0, 1.0])
        Expecting nothing
        ok
        Trying:
            expand_dis = 2.0
        Expecting nothing
        ok
        Trying:
            all_obstacles = [Circle(11.5, 5, 3), Circle(6.5, 2.5, 2.5), Circle(4.8, 8, 3.0)]
        Expecting nothing
        ok
        Trying:
            seed_v

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ea910e7b-c28e-400f-9440-52de574ecdc8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>